In [1]:
import pandas as pd

'/home/adithan'

In [4]:
fdf = pd.read_csv('file:///home/adithan/friends_test.csv')

In [5]:
fdf.head(5)

,0,Will,33,385
0,1,Jean-Luc,26,2
1,2,Hugh,55,221
2,3,Deanna,40,465
3,4,Quark,68,21
4,5,Weyoun,59,318


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize Spark Session
spark = SparkSession.builder.appName("Friends Analysis").getOrCreate()

# Define schema for the DataFrame
schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("NumFriends", IntegerType(), True)
])


24/11/04 09:16:37 WARN Utils: Your hostname, ADITHAN-MSI-M14B11SBU resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/04 09:16:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/04 09:16:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
file_path = "file:///home/adithan/friends_test.csv"  # Update with your actual file path
friends_df = spark.read.csv(file_path, schema=schema, header=False)

In [8]:
# Convert to RDD with (age, (num_friends, 1)) pairs
age_friends_rdd = friends_df.rdd.map(lambda row: (row["Age"], (row["NumFriends"], 1)))

# Calculate the sum of friends and count for each age using reduceByKey
age_friends_totals = age_friends_rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Calculate the average by dividing the sum by the count
average_friends_by_age = age_friends_totals.mapValues(lambda total_count: total_count[0] / total_count[1])

In [9]:
# Collect and print the result
result = average_friends_by_age.collect()
for age, avg_friends in result:
    print(f"Age: {age}, Average Friends: {avg_friends}")


Age: 33, Average Friends: 325.3333333333333
Age: 26, Average Friends: 242.05882352941177
Age: 55, Average Friends: 295.53846153846155
Age: 40, Average Friends: 250.8235294117647
Age: 68, Average Friends: 269.6
Age: 59, Average Friends: 220.0
Age: 37, Average Friends: 249.33333333333334
Age: 54, Average Friends: 278.0769230769231
Age: 38, Average Friends: 193.53333333333333
Age: 27, Average Friends: 228.125
Age: 53, Average Friends: 222.85714285714286
Age: 57, Average Friends: 258.8333333333333
Age: 56, Average Friends: 306.6666666666667
Age: 43, Average Friends: 230.57142857142858
Age: 36, Average Friends: 246.6
Age: 22, Average Friends: 206.42857142857142
Age: 35, Average Friends: 211.625
Age: 45, Average Friends: 309.53846153846155
Age: 60, Average Friends: 202.71428571428572
Age: 67, Average Friends: 214.625
Age: 19, Average Friends: 213.27272727272728
Age: 30, Average Friends: 235.8181818181818
Age: 51, Average Friends: 302.14285714285717
Age: 25, Average Friends: 197.4545454545454

In [10]:
# Stop Spark session
spark.stop()